# Exceptions

<div class="alert alert-danger">
<b>Antipattern: </b> Catching BaseException
</div>

In [ ]:
try:
    do_dangerous()
except:  # catch everything, even KeyboardInterrupt
    pass

In [ ]:
try:
    do_dangerous()
except BaseException:  # same as above
    pass

**Problem**: 
* Not all exceptions are bugs — some are control signals (e.g., `KeyboardInterrupt` when you press `Ctrl+C`).
* A bare `except:` catches *everything*, including those signals.
* This can make your program **impossible to stop** without force-killing it.

**Example:**

```python
def buggy_worker():
    # Bug: this "work" never finishes (infinite loop).
    while True:
        time.sleep(1)


try:
    buggy_worker()
except:  # KeyboardInterruptError is caught as well
    pass

do_something()  # you might not want to continue
```

### Exception chaining

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        try:
            return clicks / shows
        except ZeroDivisionError as e:
            raise ValueError('Bad banner') from e  # we want to know what went wrong before raising our exception
    except ValueError as e:
        raise OSError from e
        
ctr(0, 1)

OSError: 

### Exception cause

`raise ... from ...`

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError as e:
        raise ValueError('Bad banner') from e
try:
    ctr(0, 1)
except ValueError:
    print("catch value error")
except ZeroDivisionError:
    print("catch zero div")


catch value error


Context suppression

In [ ]:
def ctr(shows, clicks):
    """Returns banner click-through rate"""
    try:
        return clicks / shows
    except ZeroDivisionError as e:
        raise ValueError('Bad banner') from None
ctr(0, 1)

ValueError: Bad banner

### Exception internals

In [ ]:
try:
    raise ValueError(1, 2, 3)
except Exception as e:
    exc = e

In [ ]:
exc.args  # constructor arguments

(1, 2, 3)

In [ ]:
exc.__cause__  # exception cause, set by `raise EXC from CAUSE`
exc.__context__  # last caught exception, for exception chains
exc.__traceback__

In [ ]:
#exc.with_traceback(tb)  # sets __traceback__ to a new value tb
exc.add_note("some text")
raise exc

ValueError: (1, 2, 3)

### Warnings

In [ ]:
import numpy as np

np.int32(1) / np.int32(0)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import numpy as np

try:
    np.int32(1) / np.int32(0)
except Exeption:
    print("warning is not exception")

ModuleNotFoundError: No module named 'numpy'

So even though Warnings inherit from Exception, it's still not possible to catch and handle a warning this way.

You can turn warnings into errors:

In [ ]:
import numpy as np
import warnings

warnings.filterwarnings("error")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import numpy as np
import warnings

warnings.filterwarnings("ignore")
try:
    np.int32(1) / np.int32(0)
except Exception as e:
    print(f"Exception {e!r}")

warnings.resetwarnings()

ModuleNotFoundError: No module named 'numpy'

Useful things

- `sys.exc_info()` — returns information about the currently handled exception
- The `traceback` module
- The `warnings` module

### Exception groups

Note: this is a **bonus topic** (i.e. learn it if you feel like it)

Problem: suppose we have the following function:

`run_parralel(list_of_functions, list_of_args, n_jobs=-1)`

```python
import multiprocessing as mp

def run_parralel(list_of_functions, list_of_args, n_jobs=-1):
    with mp.Pool(n_jobs) as pool:
        pool.map(list_of_functions, list_of_args)
```

Each function called inside may raise an error. Which error should we catch? How do we understand which errors happened in each function?

```python
run_parralel([lambda x: x / 0, lambda x: x + 1], [1, 2])
```

What are the problems with Exception? Can we just pack exceptions into a "super" Exception that can represent complex errors?

### ExceptionGroup

Exception cannot fully represent an independent set of errors; therefore we have ExceptionGroup, which is a tree.

In [ ]:
eg = ExceptionGroup(
     "one",
     [
         TypeError(1),
         ExceptionGroup(
             "two",
              [TypeError(2), ValueError(3)]
         ),
         ExceptionGroup(
              "three",
               [OSError(4)]
         )
    ]
)
raise eg

### ExceptionGroup handling methods: subgroup

In [ ]:
type_errors = eg.subgroup(lambda e: isinstance(e, TypeError))
raise type_errors

### ExceptionGroup handling methods: split

In [ ]:
type_errors, other_errors = eg.split(lambda e: isinstance(e, TypeError))
raise other_errors

### ExceptionGroup handling methods: except*

In [ ]:
import errno

def low_level_os_operation() -> None:
    raise ExceptionGroup(
        "subtasks",
        [
            OSError(errno.EPIPE, "Broken pipe"),
            OSError(errno.ENOENT, "No such file or directory"),
            OSError(errno.EACCES, "Permission denied"),
            ValueError("bad value")
        ]
    )
    
try:
    low_level_os_operation()
except* OSError as errors:
    exc = errors.subgroup(lambda e: isinstance(e, OSError) and e.errno != errno.EPIPE)
    if exc is not None:
        raise exc from None
except* ValueError as errors:
    raise errors from None

# Input/Output streams

### Stream redirection

In [ ]:
%%writefile io_example.py

import sys

sys.stdout.write('Hello, world!\n')
sys.stderr.write('Error!')

Writing io_example.py


In [ ]:
!python io_example.py

Hello, world!
Error!

In [ ]:
%%bash
echo "redirect stdout to logs.txt"
python io_example.py > logs.txt


redirect stdout to logs.txt


Error!

In [ ]:
%%bash
echo "logs.txt content:"
cat logs.txt


logs.txt content:


Hello, world!


In [ ]:
%%bash
echo "redirect stderr to logs.txt"
python io_example.py 2> logs.txt


redirect stderr to logs.txt
Hello, world!


In [ ]:
%%bash
echo "logs.txt content:"
cat logs.txt

logs.txt content:
Error!

Note: separation of stdout and stderr allows you to distinguish between output (e.g. valid data) and error messages.

### UTF-8

* Variable-length encoding of Unicode:
  * ASCII (U+0000–U+007F): **1 byte**
  * Many Latin/European chars: **2 bytes**
  * Most other BMP: **3 bytes**
  * Emojis/rare symbols: **4 bytes**
* Never assume “1 char = 1 byte”.


In [ ]:
print("A".encode("utf-8"))  # b'A' is a byte, not character
print("é".encode("utf-8"))
print("€".encode("utf-8"))
print("🐍".encode("utf-8"))

In [ ]:
print(list("A".encode("utf-8")))
print(list("é".encode("utf-8")))
print(list("€".encode("utf-8")))
print(list("🐍".encode("utf-8")))
# one more 4 byte character
print(list("💡".encode("utf-8")))
print(list("🀄".encode("utf-8")))


You can read more on UTF-8 encoding [here](https://medium.com/free-code-camp/a-beginner-friendly-guide-to-unicode-d6d45a903515).

In [ ]:
message = "Secret message: 💡"

with open("secret_message.txt", "w", encoding="utf-8") as f:
    f.write(message)

with open("secret_message.txt", "r", encoding="utf-16") as f:
    print(f.read())

Note: use proper encoding; most of the time it is UTF-8

### Mini-bonus: `seek`

What if we need to go back to the beginning of a file we've read? Do we have to reopen it?

In [ ]:
!cat some_text.txt

In [ ]:
with open("some_text.txt", "r") as f:
    first_line = f.readline()
    print(first_line)
    
    # what if I want to read the first line again?
    f.seek(0)
    print(f.readline())

    # one more time
    f.seek(0)
    print(f.readline())

In [ ]:
import io

with open("some_text.txt", "r") as f:
    f.seek(-10, io.SEEK_END)  # oops...
    print(f.read())

Q: Why can't I go back? \
A: Imagine you have a symbol represented as 4 bytes (e.g. 💡). \
When you go back, you don't want to read the whole file. \
However, without reading the whole file, you can't know whether the current byte is one symbol or part of bytes sequence representing another symbol.

# Namespaces

Interesting: Each scope doesn't change during the execution of the program. \
In other words, namespaces are determined before the execution of the program.

In [1]:
globals_before = globals()
x = 1
def foo():
    pass
globals_after = globals()
new_keys = list(set(globals_after.keys()) - set(globals_before.keys()))
print(new_keys)

[]


In [2]:
def main():
    locals_before = locals()
    x = 1
    def foo():
        pass
    locals_after = locals()
    new_keys = list(set(locals_after.keys()) - set(locals_before.keys()))
    print(new_keys)

main()

[]


### `global` + `nonlocal`

##### global 

In [ ]:
global_var = 'global_var'

def func():
    global global_var
    global_var = 'global_var_modified'
    print(locals())
    print(global_var)

func()
print(global_var)

<div class="alert alert-danger">
<b>Antipattern: </b> using global
</div>

In [ ]:
def func():
    # global can create
    global new_global_var
    new_global_var = 'new_global_var'
    print(new_global_var)

func()
print(new_global_var)

### Problem

We want to use `outer` var in `inner`

In [ ]:
def outer():
    var = 'outer'

    def inner():
        global var
        var = 'inner'
        print('from inner:', var)

    inner()
    print('from outer:', var)

outer()
print('from global:', var)

##### Solution: nonlocal

In [ ]:
def outer():
    var = 'outer'

    def inner():
        nonlocal var
        var = 'inner'
        print('from inner :', var)

    inner()    
    print('from outer :', var)

var = 'global'
outer()
print('from global:', var)

<div class="alert alert-danger">
<b>Antipattern: </b> using nonlocal
</div>

### Python Types to JSON

What about Decimal, complex, datetime, ...? We want to store them too.

In [ ]:
from decimal import Decimal
num = Decimal('0.1')
json.dumps(num)

TypeError: Object of type Decimal is not JSON serializable

In [ ]:
def my_encode(obj):
    if isinstance(obj, Decimal):
        return str(obj)
    raise TypeError('Unknown object type {}'.format(type(obj)))
    
print(json.dumps(num, default=my_encode))
print(json.dumps(num, default=str))

In [ ]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return str(obj)
        return json.JSONEncoder.default(self, obj)
    
    def encode(self, obj):
        res = json.JSONEncoder.encode(self, obj)
        if isinstance(obj, list):
            return 'formatted:{}'.format(res)
        return res

data = ['hello world', Decimal('1.23'), [1.1234, 2, 3]]

print(json.dumps(data, cls=MyEncoder))

How to load Decimal, complex, datetime, ...?

In [ ]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return {'__Decimal__': str(obj)}
        return json.JSONEncoder.default(self)
    
def as_Decimal(dct):
    val = dct.get('__Decimal__')
    if val is not None:
        return Decimal(val)
    return dct

a = [Decimal('0.1'), Decimal('0.001')]
a_json = json.dumps(a, cls=DecimalEncoder)
print(a)
print(a_json)
b = json.loads(a_json, object_hook=as_Decimal)
print(b)

In [ ]:
json_str = '[0.01, 0.001]'
a = json.loads(json_str, parse_float=Decimal)
print(a)

Q: How to make it more convenient? <br>
A: Use other modules for working with json / additional modules.

Useful modules:
* dataclasses-json

In [ ]:
import simplejson

a = [0.1, Decimal('0.001')]
a_json = simplejson.dumps(a)
print(a_json)

a_parsed = simplejson.loads(a_json)
print(a_parsed, type(a_parsed[1]))

a_parsed_dec = simplejson.loads(a_json, use_decimal=True)
print(a_parsed_dec, type(a_parsed_dec[0]))

### Useful dump/dumps arguments

Note: this is a **bonus topic** 

`json.dump(obj, fp, *, skipkeys=False, ensure_ascii=True, check_...ent=None, separators=None, default=None, sort_keys=False, **kw)`

`json.dumps(obj, *, skipkeys=False, ensure_ascii=True, check_cir...ent=None, separators=None, default=None, sort_keys=False, **kw)`

...

Otherwise `locale.getpreferredencoding()` will be used.

In [ ]:
# indent + sort_keys

data = [
    {
        'name': 'Max',
        'age': 20,
    },
    {
        'name': 'Alex',
        'age': 31,
    }
]

print(json.dumps(data))
print(json.dumps(data, indent=2, sort_keys=True))

In [ ]:
# ensure_ascii - json.dump only
msg = 'Hello world!'

with open('file_ascii.txt', 'w') as fout:
    json.dump(msg, fout)
!cat file_ascii.txt
!echo "\n"

with open('file_utf8.txt', 'w', encoding='utf8') as fout:
    json.dump(msg, fout, ensure_ascii=False)
!cat file_utf8.txt

In [ ]:
with open('file_utf8.txt', encoding='utf8') as fin:
    print(json.load(fin))

It's better to set the encoding when `ensure_ascii=False`.

Otherwise `locale.getpreferredencoding()` will be used.

In [ ]:
# allow_nan

num = float('inf')
print(json.dumps(num))
print(json.dumps(num, allow_nan=False))

**Useful load/loads arguments**

`json.load(fp, *, cls=None, object_hook=None, parse_float=None, ...se_int=None, parse_constant=None, object_pairs_hook=None, **kw)`

`json.loads(s, *, cls=None, object_hook=None, parse_float=None, ...se_int=None, parse_constant=None, object_pairs_hook=None, **kw)`

In [ ]:
json.loads('{"foo": "bar"}', object_pairs_hook=print)
json.loads('{"foo": "bar"}', object_hook=print)